In [ ]:
!pip install git+https://github.com/LIAAD/yake  #Установка библиотеки YAKE из гитхаба

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-c5x8f5m0
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-c5x8f5m0
  Resolved https://github.com/LIAAD/yake to commit 0fa58cceb465162b6bd0cab7ec967edeb907fbcc
  Preparing metadata (setup.py) ... done
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=61997 sha256=8b916d70056267d51163fea5cd7636bed3bd4a9ea740d8a362e09c0d90742e0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-gmwj6ecr/wheels/06/e6/1c/8f614adcd4b053020d672b9cbc5ef1166407755a4b71adea0d
Successfully built yake


In [ ]:
import os
import zipfile
import yake

1. Загрузить и распаковать архив

In [ ]:
from google.colab import files
uploaded = files.upload()
#Чтобы исключить ошибки, лучше загрузить ZIP через код

Saving IMS2013-2024-20250413T075016Z-001 (1).zip to IMS2013-2024-20250413T075016Z-001 (1).zip


In [ ]:
zip_path = "IMS2013-2024-20250413T075016Z-001 (1).zip" #прописываем путь до него
extract_dir = "IMS_extracted" #создаем папку, куда мы хотим распаковать наш архив

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir) #извлекаем все файлы из архива в папку IMS_extracted, которая появится слева в разделе Files.

2. Прошлые опыты показали, что YAKE! лучше всего справляется с КВ. Устанавливаем и приминеям его


In [36]:
language = "ru"
language_en = "en"
max_ngram_size = 2
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

# Фильтрация по количеству слов (1 или 2)
def filter_keywords(keywords, min_n=1, max_n=2):
    return [kw for kw in keywords if min_n <= len(kw[0].split()) <= max_n][:10]  # берём максимум 10

kw_extractor_ru = yake.KeywordExtractor(
    lan=language,
    n=max_ngram_size,
    windowsSize=windowSize,
    top=numOfKeywords,
    dedupLim=deduplication_threshold,
    dedupFunc=deduplication_algo,
    features=None
)

kw_extractor_eng = yake.KeywordExtractor(
    lan=language_en,
    n=max_ngram_size,
    windowsSize=windowSize,
    top=numOfKeywords,
    dedupLim=deduplication_threshold,
    dedupFunc=deduplication_algo,
    features=None
)

In [37]:
raw_keywords = kw_extractor_ru.extract_keywords(text)
filtered_keywords = filter_keywords(raw_keywords)

В архиве все аннотации хранятся в txt файлах, которые заканчиваются на Abstract_rus.txt
Обработаем их, но учтем опечатку Anstract_rus.txt

И не забываем про английский язвк



In [38]:
results_ru = {}
results_en = {} #Ключами будут имена файлов. Значениями — 10 ключевых слов, найденных в каждом файле.

for root, dirs, files in os.walk(extract_dir): #цикл, который обходит все папки и файлы внутри extract_dir (распаковка ZIP)
    for file_name in files: #Внутри каждой папки обрабатываем каждый файл по очереди
        file_path = os.path.join(root, file_name) #Собираем полный путь к файлу, чтобы потом его открыть.

        if file_name.endswith("Abstract_rus.txt") or file_name.endswith("Anstract_rus.txt"):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                keywords = kw_extractor_ru.extract_keywords(text) #Применяем алгоритм YAKE к тексту, чтобы найти ключевые слова
                results_ru[file_name] = [kw[0] for kw in keywords] # Сохраняем ключевые слова в словарь results под именем текущего файла.

        elif file_name.endswith("Abstract_eng.txt") or file_name.endswith("Anstract_eng.txt"):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                keywords = kw_extractor_en.extract_keywords(text) #Применяем алгоритм YAKE к тексту, чтобы найти ключевые слова
                results_en[file_name] = [kw[0] for kw in keywords]

In [39]:
print(" Русские аннотации:")
for filename, kws in results_ru.items():
    print(f"\n{filename}:")
    for kw in kws:
        print(f" - {kw}")

# Результаты по английским аннотациям
print("\nАнглийские аннотации:")
for filename, kws in results_en.items():
    print(f"\n{filename}:")
    for kw in kws:
        print(f" - {kw}")

 Русские аннотации:

Vybornaya_IMS_2024_Abstract_rus.txt:
 - предложных конструкций
 - статье описываются
 - характеристики соотношения
 - исследуется синтаксическая
 - синтаксическая неоднозначность
 - синтаксически неоднозначных
 - неоднозначных предложений
 - неоднозначность предложных
 - русском языке
 - неоднозначных

Belkin_IMS_2024_Abstract_rus.txt:
 - Пользовательский фидбек
 - частности поликлиниками
 - проблемных аспектов
 - фидбек является
 - является ценным
 - ценным источником
 - источником информации
 - пользовательских отзывов
 - отзывов
 - оказываемых медицинскими

MitrofanovaAdamova_IMS_2024_Abstract_rus.txt:
 - Статья посвящена
 - корпусной лингвистике
 - лингвистики СПбГУ
 - посвящена проблемам
 - проблемам разработки
 - разработки корпуса
 - кафедре математической
 - математической лингвистики
 - корпуса статей
 - Статья

Sukhan_IMS_2024_Abstract_rus.txt:
 - корпусной лингвистике
 - лингвистики СПбГУ
 - статье представлены
 - представлены результаты
 - результаты пр

In [40]:
import pandas as pd

# Список всех уникальных имён файлов
all_files = set(results_ru.keys()).union(results_en.keys())

# Формируем таблицу
rows = []
for file_name in all_files:
    ru_keywords = results_ru.get(file_name, [])
    en_keywords = results_en.get(file_name, [])
    rows.append({
        "Имя файла": file_name,
        "Ключевые слова (рус)": ", ".join(ru_keywords),
        "Ключевые слова (англ)": ", ".join(en_keywords)
    })

# Создание DataFrame
df_keywords = pd.DataFrame(rows)

# Сохранение в CSV
df_keywords.to_csv("keywords_extracted.csv", index=False, encoding="utf-8-sig")

print("CSV-файл успешно создан: keywords_extracted.csv")

CSV-файл успешно создан: keywords_extracted.csv
